In [49]:
tot = 1/100
for i in enumerate(list(range(1,101))[::-1]:
    tot = tot[

In [41]:
import sqlite3

In [40]:
db = './scrapedata.db'
con = sqlite3.connect(db)
cur = con.cursor()
for d in ["nytimes", "bonappetit", "epicurious", "food52", "seriouseats", 
          "liquor", "allrecipes", "smittenkitchen", "foodandwine", "delish"]:
    cur.execute("SELECT Count() FROM recipes WHERE url LIKE ? ", ('%'+d+'%',))
    print(d, ':', cur.fetchone()[0])
print("total : ", cur.execute("SELECT Count() FROM recipes").fetchone()[0])

nytimes : 18461
bonappetit : 8963
epicurious : 22271
food52 : 22997
seriouseats : 2006
liquor : 5567
allrecipes : 18214
smittenkitchen : 362
foodandwine : 5490
delish : 3114
total :  107443


In [5]:
# cur.execute("SELECT title, img, url FROM recipes WHERE ingredients MATCH '%mayo% AND "
#  "(mayo)' ORDER BY bm25(recipes)").fetchall()
# con.commit()

In [60]:
%time
for i in range(1,101):
    if i % 5 ==0 and i % 3 == 0:
        print("CracklePop")
    elif i % 3 == 0:
        print("Crackle")
    elif i % 5 == 0:
        print("Pop")
    else:
        print(i)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs
1
2
Crackle
4
Pop
Crackle
7
8
Crackle
Pop
11
Crackle
13
14
CracklePop
16
17
Crackle
19
Pop
Crackle
22
23
Crackle
Pop
26
Crackle
28
29
CracklePop
31
32
Crackle
34
Pop
Crackle
37
38
Crackle
Pop
41
Crackle
43
44
CracklePop
46
47
Crackle
49
Pop
Crackle
52
53
Crackle
Pop
56
Crackle
58
59
CracklePop
61
62
Crackle
64
Pop
Crackle
67
68
Crackle
Pop
71
Crackle
73
74
CracklePop
76
77
Crackle
79
Pop
Crackle
82
83
Crackle
Pop
86
Crackle
88
89
CracklePop
91
92
Crackle
94
Pop
Crackle
97
98
Crackle
Pop


In [20]:
# db = './test.db'
# con = sqlite3.connect(db)
# cur = con.cursor()
# cur.execute("CREATE VIRTUAL TABLE IF NOT EXISTS recipes USING FTS5(title, img, url, ingredients)")
# db2 = './scrapedata.db'
# con2 = sqlite3.connect(db2)
# cur2 = con2.cursor()
# for i in cur2.execute("SELECT * from recipes").fetchall():
#     cur.execute("INSERT INTO recipes VALUES (?, ?, ?, ?)", 
#             (i[0],i[1],i[2],i[3]))